# SQuAD-Multitask-QuestionAnswer-Generation

In [1]:
# !nvidia-smi

In [2]:
# import sys

# IN_COLAB = 'google.colab' in sys.modules
# RUN_TRAINING_CELLS = IN_COLAB

# EXPERIMENT_NAME = 'SQuAD-Multitask-QuestionAnswer-Generation/'
# DRIVE_FOLDER_LOCATION = '/content/drive/My Drive/QG-Colab/Github/' + EXPERIMENT_NAME

In [3]:
# # Mounting google drive
# if IN_COLAB:
#     from google.colab import drive

#     drive.mount('/content/drive', force_remount=True)

## Environment setup
Setting up Google drive as working directory and installing packages.

In [4]:
# from datasets import load_dataset

# dataset = load_dataset("ehovy/race", "all")

In [5]:
# # Using my own Google Drive during the experiment to save all checkpoints and training logs.

# if IN_COLAB:
#     # Adapted from:  https://robertbrucecarter.com/writing/2020/06/setting-your-working-directory-to-google-drive-in-a-colab-notebook/
#     import os 

#     def create_and_set_working_directory(path: str):
#         # check if your project folder exists. if not, it will be created.
#         if os.path.isdir(path) == False:
#             os.mkdir(path)
#             print(path + ' did not exist but was created.')

#         # change the OS to use your project folder as the working directory
#         os.chdir(path)

#         print('Working directory changed to: \n' + path)

#     create_and_set_working_directory(DRIVE_FOLDER_LOCATION)
#     !pwd

In [6]:
# Install packages
# if IN_COLAB:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers

In [7]:
# Import packages
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np

import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

In [8]:
pl.seed_everything(42)

42

## Dataset
Using the NQG paper datasplit of the SQuAD v1 dataset. 

### Squad 
Reading a version of the SQuAD dataset where the there is a row for each question in the dataset.

In [9]:
# # Download squad files 
# if IN_COLAB:
#     !gdown --id 1bJylzAN7ocPTXp_ow-nLE4-hej6c68Vy #train_df.csv
#     !gdown --id 1hNJMOTVVKB--btCf3BLPc3frkcppw6fB #dev_df.csv

In [10]:
import pandas as pd
import json

# Load the JSON file into a Python dictionary
with open('/kaggle/input/squad-2/train-v2.0.json', 'r', encoding='utf-8') as f:
    squad_data = json.load(f)

# Extract paragraphs, questions, and answers from the SQuAD data
paragraphs = []
questions = []
answers = []

for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            for answer in qa.get('answers', []):
                start = answer['answer_start']
                text = answer['text']
                paragraphs.append(context)
                questions.append(question)
                answers.append(text)

# Create a DataFrame from the extracted data
train_df = pd.DataFrame({'context': paragraphs, 'question': questions, 'answer_text': answers})

# Now you have your DataFrame 'squad_df' containing the SQuAD 2.0 training dataset
train_df

,context,question,answer_text
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s
...,...,...,...
86816,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon
86817,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon
86818,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk
86819,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975


In [11]:
import pandas as pd
import json

# Load the JSON file into a Python dictionary
with open('/kaggle/input/squad-2/dev-v2.0.json', 'r', encoding='utf-8') as f:
    squad_dev_data = json.load(f)

# Extract paragraphs, questions, and answers from the SQuAD development data
dev_paragraphs = []
dev_questions = []
dev_answers = []

for article in squad_dev_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            for answer in qa.get('answers', []):
                start = answer['answer_start']
                text = answer['text']
                dev_paragraphs.append(context)
                dev_questions.append(question)
                dev_answers.append(text)

# Create a DataFrame from the extracted data
dev_df = pd.DataFrame({'context': dev_paragraphs, 'question': dev_questions, 'answer_text': dev_answers})

# Now you have your DataFrame 'squad_dev_df' containing the SQuAD 2.0 development dataset
dev_df

,context,question,answer_text
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
1,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
2,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
3,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France
4,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries
...,...,...,...
20297,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20298,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20299,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
20300,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène


### NQG datasplit
Using the datasplit proposed by the NQG paper - https://github.com/xinyadu/nqg 

In [12]:
test_df = train_df[:11877]
train_df = train_df[11877:]
train_df.shape

(74944, 3)

In [13]:
dev_df.iloc[0]['context']

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

#Training with a [ MASK] 

## Pytorch Lightning Dataset 
Using a special `sep` token to separate the parts we want to predict and a `MASK` token to pass instead of the target answer when we don't want to do answer-aware question generation.

In [14]:
SEP_TOKEN = '<sep>'
MASKING_CHANCE = 0.3 #30% chance to replace the answer with '[MASK]'

In [15]:
class QGDataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int,
        target_max_token_len: int
        ):

        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        if np.random.rand() > MASKING_CHANCE:
            answer = data_row['answer_text']
        else:
            answer = '[MASK]'

        source_encoding = tokenizer(
            '{} {} {}'.format(answer, SEP_TOKEN, data_row['context']),
            max_length= self.source_max_token_len,
            padding='max_length',
            truncation= True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )
    
        target_encoding = tokenizer(
            '{} {} {}'.format(data_row['answer_text'], SEP_TOKEN, data_row['question']),
            max_length=self.target_max_token_len,
            padding='max_length',
            truncation = True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        labels = target_encoding['input_ids']  
        labels[labels == 0] = -100

        return dict(
            answer_text = data_row['answer_text'],
            context = data_row['context'],
            question = data_row['question'],
            input_ids = source_encoding['input_ids'].flatten(),
            attention_mask = source_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
            )

## Pytorch Lightning DataModule

In [16]:
import pytorch_lightning as pl

class QGDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df: pd.DataFrame,
        val_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size,
        source_max_token_len: int,
        target_max_token_len: int
    ): 
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dataset = QGDataset(self.train_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
            self.val_dataset = QGDataset(self.val_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
        if stage == 'test' or stage is None:
            self.test_dataset = QGDataset(self.test_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self): 
        return DataLoader(self.val_dataset, batch_size=1, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=2)


#### Testing DataModule

In [17]:
# data_module = QGDataModule(train_df, dev_df, test_df, tokenizer, 2, 128, 64)
# data_module.setup()

## Hyperparameters

In [18]:
MODEL_NAME = 't5-small'
SOURCE_MAX_TOKEN_LEN = 300
TARGET_MAX_TOKEN_LEN = 80

N_EPOCHS = 5
BATCH_SIZE = 16
LEARNING_RATE = 0.0001

In [19]:
DF_TAKE_PERCENTAGE = 1

TAKE_TRAIN = int(len(train_df) * DF_TAKE_PERCENTAGE)
TAKE_DEV = int(len(dev_df) * DF_TAKE_PERCENTAGE)
TAKE_TEST = int(len(test_df) * DF_TAKE_PERCENTAGE)

print('Taking', DF_TAKE_PERCENTAGE * 100, '%')
print(TAKE_TRAIN, 'of', len(train_df))
print(TAKE_DEV, 'of', len(dev_df))
print(TAKE_TEST, 'of', len(test_df))

Taking 100 %
74944 of 74944
20302 of 20302
11877 of 11877


### Initializing training module

#### Setting DataModule

In [20]:
print(train_df[:TAKE_TRAIN].shape, dev_df[:TAKE_DEV].shape, test_df[:TAKE_TEST].shape)

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
print('tokenizer len before: ', len(tokenizer))
tokenizer.add_tokens(SEP_TOKEN)
print('tokenizer len after: ', len(tokenizer))
TOKENIZER_LEN = len(tokenizer)

data_module = QGDataModule(train_df[:TAKE_TRAIN], dev_df[:TAKE_DEV], test_df[:TAKE_TEST], tokenizer, BATCH_SIZE, SOURCE_MAX_TOKEN_LEN, TARGET_MAX_TOKEN_LEN)
data_module.setup()

(74944, 3) (20302, 3) (11877, 3)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

tokenizer len before:  32100
tokenizer len after:  32101


#### Setting Model

In [21]:
class QGModel(pl.LightningModule):
    def __init__(self, data_module):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN)
        self.data_module = data_module
        
    def test_dataloader(self):
        return self.data_module.test_dataloader()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

#### Setting trainer

In [22]:

checkpoint_callback = ModelCheckpoint(
        dirpath='checkpoints',
        filename='best-checkpoint',
        save_top_k=-1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

In [23]:
trainer = pl.Trainer(
        max_epochs=N_EPOCHS,
    )

## Training

In [24]:
%load_ext tensorboard

In [25]:
%tensorboard --logdir ./lightning_logs

In [26]:
model = QGModel(data_module)
# model = QGModel.load_from_checkpoint('checkpoints/best-checkpoint-v42.ckpt')

trainer.fit(model, data_module)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

2024-04-14 06:07:01.614732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 06:07:01.614801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 06:07:01.616339: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [27]:
# Save the entire model
torch.save(model, 'qg_model.pth')


**JavaScript to prevent from shutting down.**

function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

setInterval(ConnectButton,60000);

In [28]:
trainer.test()

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.4729632139205933     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.4729632139205933}]

## Evaluate

### Load model

In [29]:
# checkpoint_path = 'checkpoints/best-checkpoint-v9.ckpt'

# best_model = QGModel.load_from_checkpoint(checkpoint_path)
# best_model.freeze()
# best_model.eval()

# print()

### Common functions

In [30]:
# SEP_TOKEN

In [31]:
# def generate(qgmodel: QGModel, answer: str, context: str) -> str:
#     source_encoding = tokenizer(
#         '{} {} {}'.format(answer, SEP_TOKEN, context),
#         max_length=SOURCE_MAX_TOKEN_LEN,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         add_special_tokens=True,
#         return_tensors='pt'
#     )

#     generated_ids = qgmodel.model.generate(
#         input_ids=source_encoding['input_ids'],
#         attention_mask=source_encoding['attention_mask'],
#         num_beams=1,
#         max_length=TARGET_MAX_TOKEN_LEN,
#         repetition_penalty=2.5,
#         length_penalty=1.0,
#         early_stopping=True,
#         use_cache=True
#     )

#     preds = {
#         tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
#         for generated_id in generated_ids
#     }

#     return ''.join(preds)

In [32]:
# def show_result(generated: str, answer: str, context:str, original_question: str = ''):
#     print('Generated: ', generated)
#     if original_question:
#         print('Original : ', original_question)

#     print()
#     print('Answer: ', answer)
#     print('Conext: ', context)
#     print('-----------------------------')

### View results manually 

In [33]:
# sample_question = test_df.iloc[42]

# generated = generate(best_model, sample_question['answer'], sample_question['context'])
# show_result(generated, sample_question['answer'], sample_question['context'], sample_question['question'])

In [34]:
# context = 'Oxygen is the chemical element with the symbol O and atomic number 8.'
# answer = 'Oxygen'

# generated = generate(best_model, answer, context)

# show_result(generated, answer, context)

In [35]:
# context = 'Oxygen is the chemical element with the symbol O and atomic number 8.'
# answer = 'Oxygen'
# input_answer = '[MASK]'

# generated = generate(best_model, input_answer, context)

# show_result(generated, answer, context)

#### Answer-aware question generation

In [36]:
# for i in range(len(test_df[:10])):
#     context = test_df.iloc[i]['context']
#     answer = test_df.iloc[i]['answer']
    
#     generated = generate(best_model, answer, context)
    
#     show_result(generated, answer, context, test_df.iloc[i]['question'])

#### Generating both answer and question

In [37]:
# for i in range(len(test_df[:10])):
#     context = test_df.iloc[i]['context']
#     original_answer = test_df.iloc[i]['answer']
#     input_answer = '[MASK]'
    
#     generated = generate(best_model, input_answer, context)
    
#     show_result(generated, original_answer, context, test_df.iloc[i]['question'])

# Loading model for evaluation 

In [38]:
# MODEL_NAME = 't5-small'
# SOURCE_MAX_TOKEN_LEN = 300
# TARGET_MAX_TOKEN_LEN = 80
# SEP_TOKEN = '<sep>'

# tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
# tokenizer.add_tokens(SEP_TOKEN)
# TOKENIZER_LEN = len(tokenizer)

In [39]:
# class QGModel(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
#         self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

#     def forward(self, input_ids, attention_mask, labels=None):
#         output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         return output.loss, output.logits

#     def training_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, output = self(input_ids, attention_mask, labels)
#         self.log('train_loss', loss, prog_bar=True, logger=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, output = self(input_ids, attention_mask, labels)
#         self.log('val_loss', loss, prog_bar=True, logger=True)
#         return loss

#     def test_step(self, batch, batch_idx):
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']
#         loss, output = self(input_ids, attention_mask, labels)
#         self.log('test_loss', loss, prog_bar=True, logger=True)
#         return loss
  
#     def configure_optimizers(self):
#         return AdamW(self.parameters(), lr=LEARNING_RATE)

In [40]:
# checkpoint_path = 'checkpoints/best-checkpoint-v9.ckpt'

# best_model = QGModel.load_from_checkpoint(checkpoint_path)
# best_model.freeze()
# best_model.eval()

# print()

In [41]:
# def generate(qgmodel: QGModel, answer: str, context: str) -> str:
#     source_encoding = tokenizer(
#         '{} {} {}'.format(answer, SEP_TOKEN, context),
#         max_length=SOURCE_MAX_TOKEN_LEN,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         add_special_tokens=True,
#         return_tensors='pt'
#     )

#     generated_ids = qgmodel.model.generate(
#         input_ids=source_encoding['input_ids'],
#         attention_mask=source_encoding['attention_mask'],
#         num_beams=1,
#         max_length=TARGET_MAX_TOKEN_LEN,
#         repetition_penalty=1.0,
#         length_penalty=1.0,
#         early_stopping=True,
#         use_cache=True
#     )

#     preds = {
#         tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         for generated_id in generated_ids
#     }

#     return ''.join(preds)

In [42]:
# def show_result(generated: str, answer: str, context:str, original_question: str = ''):
#     print('Generated: ', generated)
#     if original_question:
#         print('Original : ', original_question)

#     print()
#     print('Answer: ', answer)
#     print('Conext: ', context)
#     print('-----------------------------')

In [43]:
# sample_question = test_df.iloc[42]

# generated = generate(best_model, sample_question['answer'], sample_question['context'])
# show_result(generated, sample_question['answer'], sample_question['context'], sample_question['question'])